In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")

In [14]:
import pandas as pd
import geopandas as gpd

from src.modeling.utils import temporal_train_test_split
from src.modeling.bayesian_models import create_st_blr

from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path
from src.modeling.encodings import convert_aspect_to_cardinal_direction

In [8]:
paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

### Read in data

In [10]:
# read in training data
training_data = gpd.read_file(paths["training_data"]["subset"])
training_data.date = pd.to_datetime(training_data.date)


In [11]:
# read in NUTS data
nuts_austria = gpd.read_file(paths["nuts_data"])

### Data Cleaning & Preparation

In [15]:
training_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        921 non-null    datetime64[ns]
 1   Pufferradi  744 non-null    object        
 2   fire        921 non-null    int64         
 3   ffmc        921 non-null    float64       
 4   farmyard_d  921 non-null    float64       
 5   hiking_ds   921 non-null    float64       
 6   forest_ds   921 non-null    float64       
 7   rail_dens   921 non-null    float64       
 8   elevation   921 non-null    float64       
 9   slope       921 non-null    float64       
 10  aspect      921 non-null    float64       
 11  foresttype  886 non-null    float64       
 12  pop_dens    921 non-null    float64       
 13  geometry    921 non-null    geometry      
dtypes: datetime64[ns](1), float64(10), geometry(1), int64(1), object(1)
memory usage: 100.9+ KB


In [13]:
training_data

# rename columns
training_data.rename(columns={"farmyard_d": "farmyard_density", "hiking_ds": "hikingtrail_density", "forest_ds": "forestroad_density", "rail_dens": "railway_density", "foresttype": "forest_type", "pop_dens": "population_density"})

# create nuts2 column

# create season column

# missing data (especially forest type)

# categorize aspect 

,date,Pufferradi,fire,ffmc,farmyard_d,hiking_ds,forest_ds,rail_dens,elevation,slope,aspect,foresttype,pop_dens,geometry
0,2021-05-27,300,1,82.984094,0.0,0.000000,0.000000,0.000000,353.587067,14.855182,208.116684,2.0,1186.000000,POINT (609377.781 445800.096)
1,2021-05-21,50,1,88.696592,0.0,0.000000,293.121563,255.336817,303.168518,0.601211,95.196953,2.0,1.453001,POINT (613414.750 434937.521)
2,2021-05-21,250,1,88.569387,0.0,0.000000,79.307386,0.000000,321.198608,0.510599,45.291309,6.0,0.000000,POINT (611564.181 433471.046)
3,2021-05-11,50,1,91.831419,0.0,0.000000,0.000000,0.000000,541.701843,1.323429,228.802277,6.0,1234.000000,POINT (550216.159 397819.592)
4,2021-05-11,50,1,88.612258,0.0,0.000000,0.000000,0.000000,722.024597,30.482531,186.500824,4.0,209.689070,POINT (281489.140 390784.067)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,2018-08-15,None,0,87.461530,0.0,0.000000,0.000000,0.000000,584.361877,7.295747,164.582504,5.0,8.000000,POINT (550763.375 521182.049)
917,2016-03-15,None,0,50.851050,0.0,41.883956,0.000000,0.000000,282.322083,7.166595,42.359444,5.0,0.000000,POINT (671127.712 475160.391)
918,2015-04-11,None,0,85.047707,0.0,0.000000,0.000000,0.000000,1719.081177,22.102749,269.180054,2.0,0.000000,POINT (580854.459 425888.616)
919,2016-11-19,None,0,66.894841,0.0,0.000000,0.000000,0.000000,1427.435425,9.860193,207.258575,1.0,0.000000,POINT (524062.413 415188.230)
